In [ ]:
from p2pfl.node import Node
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía

        
test_convergence((2,2))

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpwb3qaumo
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpwb3qaumo/_remote_module_non_sriptable.py
INFO:root:Nodo a la escucha en localhost 56332
INFO:root:Nodo a la escucha en localhost 56333
INFO:root:Conexión aceptada con ('localhost', 56333)
INFO:root:Broadcasting start learning...


Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py", line 247, in start_learning
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py", line 247

In [3]:
from p2pfl.const import HEARTBEAT_FREC, TIEMOUT
from p2pfl.node import Node
import time

n1 = Node("localhost")
n2 = Node("localhost")
n1.start()
n2.start()

def test_interrupt_train2(two_nodes):
    n1, n2 = two_nodes
    n1.connect_to(n2.host,n2.port)

    time.sleep(1) #Esperar por la asincronía

    n1.set_start_learning(99999,99999)

    time.sleep(1) #Esperar por la asincronía

    n2.set_stop_learning()
    
    time.sleep(1) #Esperar por la asincronía
    
    print(n1.round)
    print(n2.round)
    
    assert n1.round is None
    
test_interrupt_train2((n1,n2))

INFO:root:Nodo a la escucha en localhost 65058
INFO:root:Nodo a la escucha en localhost 65059
INFO:root:Conexión aceptada con ('localhost', 65058)
INFO:root:Broadcasting start learning...
INFO:root:Training...


GPU available: False, used: False


INFO:root:Training...


TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
HPU available: False, using: 0 HPUs

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params si

Train: 54000 Val:6000 Test:10000Train: 54000 Val:6000 Test:10000



Sanity Checking: 0it [00:00, ?it/s]

Sanity Checking: 0it [00:00, ?it/s]

INFO:root:Stopping learning
INFO:root:Stopping learning
None
None


In [6]:
from p2pfl.node import Node
import torch

n1 = Node("localhost")
n1.start()

sum = n1.learner.get_parameters()
print(sum)

INFO:root:Nodo a la escucha en localhost 53709
OrderedDict([('l1.weight', tensor([[-0.0287, -0.0048,  0.0294,  ..., -0.0250,  0.0217,  0.0113],
        [ 0.0220,  0.0061, -0.0328,  ...,  0.0185, -0.0317,  0.0078],
        [-0.0262,  0.0216, -0.0159,  ...,  0.0109, -0.0086, -0.0018],
        ...,
        [-0.0180, -0.0295, -0.0029,  ...,  0.0145,  0.0264, -0.0134],
        [-0.0250,  0.0020,  0.0125,  ..., -0.0091, -0.0146,  0.0017],
        [-0.0181,  0.0163, -0.0033,  ..., -0.0196,  0.0291, -0.0250]])), ('l1.bias', tensor([-0.0087, -0.0157,  0.0090, -0.0334, -0.0151,  0.0152, -0.0332, -0.0143,
         0.0033, -0.0339,  0.0156, -0.0132,  0.0216, -0.0017,  0.0076,  0.0226,
         0.0004,  0.0032, -0.0017,  0.0002,  0.0179,  0.0308, -0.0074,  0.0035,
         0.0149,  0.0118,  0.0151, -0.0157, -0.0181, -0.0106,  0.0268, -0.0154,
        -0.0269,  0.0259,  0.0116, -0.0209,  0.0097,  0.0323,  0.0036, -0.0121,
        -0.0157, -0.0216,  0.0324,  0.0271, -0.0292,  0.0356,  0.0192,  0.0357

In [7]:
sum = [torch.zeros_like(sum[layer]) for layer in sum]
print(sum)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [

In [11]:
from p2pfl.learning.agregators.fedavg import FedAvg
from p2pfl.learning.pytorch.models.mlp import MLP
from p2pfl.learning.pytorch.learners.lightninglearner import LightningLearner
from collections import OrderedDict
import torch



def test_avg_simple():

    a = OrderedDict([('a', torch.tensor(-1)), ('b', torch.tensor(-1))])
    b = OrderedDict([('a', torch.tensor(0)), ('b', torch.tensor(0))])
    c = OrderedDict([('a', torch.tensor(1)), ('b', torch.tensor(1))])

    result = FedAvg.agregate([(a,1),(b,1),(c,1)])

    for layer in b:
        assert result[layer] == b[layer]
        
test_avg_simple()

In [1]:
from p2pfl.learning.agregators.fedavg import FedAvg
from p2pfl.learning.pytorch.models.mlp import MLP
from p2pfl.learning.pytorch.learners.lightninglearner import LightningLearner
from collections import OrderedDict
import torch


def test_avg_complex():
    nl1 = LightningLearner(MLP(), None)
    params = nl1.get_parameters()
    params1 = nl1.get_parameters()
    params2 = nl1.get_parameters()

    result = FedAvg.agregate([(params,1)])

    # Check Results
    for layer in params:
        assert torch.eq(params[layer], result[layer]).all()

    for layer in params2:
        params1[layer] = params1[layer]+1
        params2[layer] = params2[layer]-1
    
    result = FedAvg.agregate([(params1,1), (params2,1)])

    # Check Results
    for layer in params:
        a = torch.trunc(params[layer]*10)
        b = torch.trunc(result[layer]*10)
        assert torch.eq(a, b).all()
        
while True:
    test_avg_complex()

KeyboardInterrupt: 

SyntaxError: invalid syntax (2964970389.py, line 1)

In [ ]:
from p2pfl.node import Node


n1 = Node("localhost")
n1.start()

n1.learner